# Training

In diesem Notebook wird das Klassifizierungs-Modell erstellt und Trainiert. Den Datensatz, welchen wir für das Training genutzt haben, haben wir von Kaggle:

https://www.kaggle.com/datasets/xhlulu/140k-real-and-fake-faces/

Der Datensatz besteht aus 265x265 Pixel großen Bildern von 70k echten Gesichtern aus der Nvidia-Datenbank "Flickr" und 70k KI-Generierten Bildern von "StyleGAN".

## Imports

- Torch - Erstellung des Modells
- Matplotlib - Darstellung der Bilder
- Tqdm - Ladebalken beim Training

In [ ]:
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt
import random
from tqdm import tqdm

## Datenset laden

Den Datensatz laden wir aus Google Drive, da es mit Colab am besten funktioniert.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/Dataset/data.zip /content
!unzip /content/data.zip

## Konstanten

Diese Konstanten wurden so optimiert, damit wir die bestmögliche Accuracy erhalten.

In [ ]:
ROOT_DIR: str = "/content/data"
TRAIN_TEST_RATIO: float = 0.2 # Anteil von Test split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Konstanten die optimiert werden sollen
IMAGE_SIZE: int = 256
BATCH_SIZE: int = 512 # Hohe Batch-Size => Gut für GPUs mit viel RAM
HIDDEN_UNITS: int = 32
LEARNING_RATE: float = 0.001
EPOCHS: int = 10

## Daten in ImageFolder speichern

Hier werden die Daten Transformiert, um sie zu normen und um Overfitting zu verhindern.

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), # Resize auf einheitliche Größe
    torchvision.transforms.RandomHorizontalFlip(), # Hälfte der Bilder horizontal spiegeln
    torchvision.transforms.RandomRotation(degrees=15), # Zufällige Rotation
    torchvision.transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)), # Zufälliges Zuschneiden
    torchvision.transforms.ToTensor(), # In Tensor umwandeln
])

In [ ]:
# Die Klassen werden Automatisch nach den Ordernamen gespeichert (fake, real)
dataset = torchvision.datasets.ImageFolder(root=ROOT_DIR, transform=transform)


## Bilder darstellen

Hier werden ein paar zufällige Bilder zur Kontrolle ausgegeben.

In [ ]:
num_images = 3

for _ in range(num_images):

    # Zufälliges Bild und Label speichern
    index = random.randint(0, len(dataset) - 1)
    img, label = dataset[index]

    # Tensor in Bild umwandeln
    img = img.permute(1, 2, 0)

    # Darstelluung mit Pyplot
    plt.figure(figsize=(3, 3))
    plt.imshow(img)
    plt.title(f"Label: {dataset.classes[label]} & Index: {index}")

## Dataloader erstellen

Hier werden Train- und Test-Dataloader erstellt, die einer effizienten Verarbeitung von Daten dienen.

In [ ]:
# Train/Test Split berechnen
total_size = len(dataset)
test_size = int(TRAIN_TEST_RATIO * total_size)
train_size = total_size - test_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# DataLoader mit Batches
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

## Ausgabe von Metadaten

In [ ]:
print(f"Länge Datensatz: {len(dataset)}\n")
print("Shapes der Tensoren:")
print(f"Labels: {dataset.classes}")

images, labels = next(iter(train_loader))

print(f"Bilder-Batch Shape: {images.shape}")
print(f"Label-Batch Shape: {labels.shape}")

## CNN-Model Klasse erstellen

Dieses Modell haben wir von https://poloclub.github.io/cnn-explainer/. Es ist aber für unsere Zwecke etwas abgeändert wurden (z.B. Dropout-Layer hinzugefügt).

In [ ]:
class CNNModel(nn.Module):
    """
    TinyVGG:
    https://poloclub.github.io/cnn-explainer/
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(in_features=hidden_units * (IMAGE_SIZE//4) * (IMAGE_SIZE//4),
                    out_features=output_shape)
        )


    def forward(self, x: torch.Tensor):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.classifier(x)
        return x

model = CNNModel(input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(dataset.classes)).to(device)

print(model)

## Loss-Function, Optimizer, Scheduler

Loss-Functions und Optimizer sind bei Torch (und auch allgemein bei Neuronalen Netzen) essentiell. Wie nutzen hier die Cross-Entropy-Lossfunction und den Adam Optimizer, da diese für CNN-Klassifizierungen optimal sind.

Der Learning-Rate-Scheduler ist optional, aber sehr hilfreich, denn dieser passt die Lernrate automatisch an, falls das Modell ein Plateu erreicht und sich nicht verbessert.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.1,
    patience=3,
    threshold=1e-4
)

## Hilfsfunktionen

Diese Funktionen haben wir von dem Online-Kurs: https://www.learnpytorch.io/. Sie sind zum Training und Testen des Modells gedacht und dienen dazu die Trainingsschleife übersichtlich zu halten.

In [ ]:
# Evaluierung des Modells nach Training

def eval_model(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn):
    """Returns a dictionary containing the results of model predicting on data_loader.

    Args:
        model (torch.nn.Module): A PyTorch model capable of making predictions on data_loader.
        data_loader (torch.utils.data.DataLoader): The target dataset to predict on.
        loss_fn (torch.nn.Module): The loss function of model.
        accuracy_fn: An accuracy function to compare the models predictions to the truth labels.

    Returns:
        (dict): Results of model making predictions on data_loader.
    """
    loss, acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            # Make predictions with the model
            y_pred = model(X)

            # Accumulate the loss and accuracy values per batch
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y_true=y,
                                y_pred=y_pred.argmax(dim=1)) # For accuracy, need the prediction labels (logits -> pred_prob -> pred_labels)

        # Scale loss and acc to find the average loss/acc per batch
        loss /= len(data_loader)
        acc /= len(data_loader)

    return {"model_name": model.__class__.__name__, # only works when model was created with a class
            "model_loss": loss.item(),
            "model_acc": acc}

In [ ]:
# Training und Testen des Modells (einmal pro Epoche)

def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    model.to(device)
    for batch, (X, y) in enumerate(data_loader):
        # Send data to GPU
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.to(device)
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode():
        for X, y in data_loader:
            # Send data to GPU
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred = model(X)

            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )

        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

    return test_loss

In [ ]:
# Berechnung der Accuracy (Genauigkeit des Modells) zum Vergleich

def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

## Trainingsschleife

Pro Epoche wird einmal Trainiert und Getestet und danach die Lernrate angepasst.

In [ ]:
EPOCHS = 100

for epoch in tqdm(range(EPOCHS)):
    print(f"Epoch: {epoch}\n---------")
    train_step(data_loader=train_loader,
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn,
        device=device
    )
    test_loss = test_step(data_loader=test_loader,
        model=model,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn,
        device=device
    )

    lr_scheduler.step(test_loss)

## Evaluierung und Speicherung

Das Modell wird zur Sicherheit einmal komplett gespeichert (veraltet) und einmal nur die Gewichte.

In [ ]:
model_results = eval_model(
    model=model,
    data_loader=test_loader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn
)
model_results

In [ ]:
def save_model(model: torch.nn.Module,
               model_name: str):
  torch.save(model.state_dict(), f"/content/drive/MyDrive/Dataset/saves/model_weights_{model_name}.pth")
  torch.save(model, f"/content/drive/MyDrive/Dataset/saves/full_model_weights_{model_name}.pth")

In [ ]:
# Speichern mit Accuracy im Namen
save_model(model, model_results["model_acc"])